<a href="https://colab.research.google.com/github/olfabre/amsProjetMaster1/blob/olivier/Generation_prenoms_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
try:
    import unidecode
except ModuleNotFoundError:
    !pip install unidecode
    import unidecode

import requests
import torch
import torch.nn as nn
from torch.autograd import Variable
import time
import math
import string
import random
import os

# Vérification GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Appareil utilisé : {device}")

# Téléchargement des données
url = "https://olivier-fabre.com/passwordgenius/russian.txt"
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)
data_path = os.path.join(data_dir, "russian.txt")

if not os.path.exists(data_path):
    print("Téléchargement des données...")
    response = requests.get(url)
    with open(data_path, 'w', encoding='utf-8') as f:
        f.write(response.text)

# Chargement des données
def unicode_to_ascii(s):
    return ''.join(
        c for c in unidecode.unidecode(s)
        if c in (string.ascii_letters + " .,;'-")
    )

def read_lines(filename):
    with open(filename, encoding='utf-8') as f:
        return [unicode_to_ascii(line.strip().lower()) for line in f]

lines = read_lines(data_path)
print(f"Nombre de prénoms : {len(lines)}")

# Division des données
random.shuffle(lines)
train_split = int(0.8 * len(lines))
valid_split = int(0.1 * len(lines))
train_lines = lines[:train_split]
valid_lines = lines[train_split:train_split + valid_split]
test_lines = lines[train_split + valid_split:]
print(f"Ensemble d'entraînement : {len(train_lines)}, Validation : {len(valid_lines)}, Test : {len(test_lines)}")

# Paramètres globaux
all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1  # EOS marker
hidden_size = 128
n_layers = 2
lr = 0.005
bidirectional = True
max_length = 20

# Fonctions utilitaires
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_letters.index(string[c])
    return tensor

def input_tensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

def target_tensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1)  # EOS
    return torch.LongTensor(letter_indexes)

def random_training_example(lines):
    line = random.choice(lines)
    input_line_tensor = input_tensor(line)
    target_line_tensor = target_tensor(line)
    return input_line_tensor, target_line_tensor

# Fonction pour afficher le temps écoulé
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return f"{m}m {s:.2f}s"

# Définition du modèle
class RNNLight(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNLight, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.bidirectional = bidirectional
        self.num_directions = 2 if self.bidirectional else 1
        self.rnn = nn.RNN(
            input_size=input_size, hidden_size=hidden_size,
            num_layers=1, bidirectional=self.bidirectional, batch_first=True
        )
        self.out = nn.Linear(self.num_directions * hidden_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        _, hidden = self.rnn(input.unsqueeze(0), hidden)
        hidden_concat = hidden if not self.bidirectional else torch.cat((hidden[0], hidden[1]), 1)
        output = self.out(hidden_concat)
        output = self.dropout(output)
        return self.softmax(output), hidden

    def init_hidden(self):
        return torch.zeros(self.num_directions, 1, self.hidden_size, device=device)

# Entraînement avec sauvegarde
def train(input_line_tensor, target_line_tensor, decoder, decoder_optimizer, criterion):
    target_line_tensor = target_line_tensor.to(device)  # Déplacement vers le bon dispositif
    hidden = decoder.init_hidden().to(device)  # Initialisation sur le bon dispositif
    decoder.zero_grad()
    loss = 0
    for i in range(input_line_tensor.size(0)):
        input_tensor = input_line_tensor[i].to(device)  # Déplacement explicite
        target_tensor = target_line_tensor[i].unsqueeze(0).to(device)  # Déplacement explicite
        output, hidden = decoder(input_tensor, hidden.detach())  # Utilisation de detach
        l = criterion(output, target_tensor)
        loss += l
    loss.backward()
    decoder_optimizer.step()
    return loss.item() / input_line_tensor.size(0)

def validation(input_line_tensor, target_line_tensor, decoder, criterion):
    with torch.no_grad():  # Pas de calcul de gradients pendant la validation
        target_line_tensor = target_line_tensor.to(device)
        hidden = decoder.init_hidden().to(device)
        loss = 0
        for i in range(input_line_tensor.size(0)):
            input_tensor = input_line_tensor[i].to(device)
            target_tensor = target_line_tensor[i].unsqueeze(0).to(device)
            output, hidden = decoder(input_tensor, hidden.detach())
            l = criterion(output, target_tensor)
            loss += l
        return loss.item() / input_line_tensor.size(0)

def training(n_epochs, train_lines, valid_lines, decoder, decoder_optimizer, criterion):
    print("\n-----------\n|  ENTRAÎNEMENT  |\n-----------\n")
    start = time.time()
    best_loss = float("inf")
    model_path = "best_model_generation_prenom.pth"

    for epoch in range(1, n_epochs + 1):
        # Entraînement
        input_line_tensor, target_line_tensor = random_training_example(train_lines)
        train_loss = train(input_line_tensor, target_line_tensor, decoder, decoder_optimizer, criterion)

        # Validation
        input_line_tensor, target_line_tensor = random_training_example(valid_lines)
        val_loss = validation(input_line_tensor, target_line_tensor, decoder, criterion)

        # Sauvegarde du meilleur modèle
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(decoder.state_dict(), model_path)
            print(f"Époch {epoch} : La perte de validation a diminué à {best_loss:.4f}. Modèle sauvegardé.")

        if epoch % 500 == 0:
            print(f"{time_since(start)} Époch {epoch}/{n_epochs}, Perte entraînement : {train_loss:.4f}, Perte validation : {val_loss:.4f}")

# Exécution principale
if __name__ == "__main__":
    decoder = RNNLight(n_letters, hidden_size, n_letters).to(device)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    n_epochs = 5000

    print("Démarrage de l'entraînement...")
    training(n_epochs, train_lines, valid_lines, decoder, decoder_optimizer, criterion)

    print("\nGénération de prénoms :")
    for letter in "ABC":
        print(sample(decoder, letter))



Appareil utilisé : cuda:0
Nombre de prénoms : 9408
Ensemble d'entraînement : 7526, Validation : 940, Test : 942
Démarrage de l'entraînement...

-----------
|  ENTRAÎNEMENT  |
-----------

Époch 1 : La perte de validation a diminué à 3.8594. Modèle sauvegardé.
Époch 2 : La perte de validation a diminué à 3.4664. Modèle sauvegardé.
Époch 4 : La perte de validation a diminué à 3.4596. Modèle sauvegardé.
Époch 6 : La perte de validation a diminué à 3.2671. Modèle sauvegardé.
Époch 9 : La perte de validation a diminué à 2.7411. Modèle sauvegardé.
Époch 20 : La perte de validation a diminué à 2.6743. Modèle sauvegardé.
Époch 33 : La perte de validation a diminué à 2.4950. Modèle sauvegardé.
Époch 51 : La perte de validation a diminué à 2.4003. Modèle sauvegardé.
Époch 57 : La perte de validation a diminué à 1.9809. Modèle sauvegardé.
Époch 67 : La perte de validation a diminué à 1.8883. Modèle sauvegardé.
Époch 87 : La perte de validation a diminué à 1.8649. Modèle sauvegardé.
Époch 146 : La